In [44]:
import pandas as pd
import math
import numpy as np
from astropy.coordinates import SkyCoord
from astropy.io import fits
import healpy as hp
import matplotlib.pyplot as plt
import time
import sys
from pixell import enmap, enplot, reproject, utils, curvedsky, wcsutils
from matplotlib import cm
from scipy.optimize import curve_fit
from scipy.linalg import sqrtm
from tqdm import tqdm
import random
import time
import multiprocessing as mp
import numpy as np
import random
import time
import multiprocessing as mp
import numpy as np

---

In [3]:
ard_not_adv = pd.read_csv('ard_not_adv.csv', skiprows = 0)

In [4]:
ard_not_adv

,Unnamed: 0,Unnamed: 0.1,amf_no,amf_ra,amf_dec,amf_z,amf_lk,amf_rh,amf_r200,amf_rc,...,red_id,whl_id,glat,glon,amf_sn,intensity100,intensity857,intensity143,intensity353,cluster
0,0,0,5.0,260.6324,32.1398,0.2252,222.1219,200.7117,1.784,0.625,...,RMJ172227.2+320757.2,J172227.2+320757,31.846666,55.602221,24.655148,-0.000048,1.978746,-0.000006,0.001062,1
1,1,1,7.0,250.1485,46.6917,0.2248,225.9874,182.3373,1.725,0.495,...,RMJ164019.8+464241.5,J164019.8+464242,41.429880,72.599428,44.674799,-0.000269,1.196847,-0.000215,0.000722,1
2,3,3,10.0,203.1873,50.5346,0.3110,164.8704,179.1883,1.663,0.422,...,RMJ133238.4+503336.0,J133252.1+503134,65.305106,107.103759,25.696075,-0.000124,1.019871,-0.000129,0.000580,1
3,4,4,12.0,350.2432,29.2029,0.5231,165.0015,174.7108,1.518,0.390,...,RMJ232104.1+291134.5,J232104.1+291134,-29.646780,100.211077,13.793623,0.000047,4.913466,0.000091,0.001929,1
4,5,5,13.0,139.4938,51.7236,0.2574,169.5809,172.6889,1.676,0.199,...,RMJ091753.4+514337.5,J091753.4+514338,43.387615,166.090009,20.839521,0.000029,1.074994,0.000060,0.000894,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21001,21533,21533,54265.0,117.7965,28.4314,0.4498,21.3133,20.0019,0.721,0.090,...,-9999,J075110.0+282537,24.760679,192.272829,0.235375,-0.000085,1.742793,-0.000074,0.000728,1
21002,21534,21534,54266.0,10.2234,16.4729,0.3325,7.5789,20.0018,0.754,0.420,...,-9999,J004043.6+162754,-46.323866,119.270217,1.128670,-0.000071,1.821485,-0.000079,0.000611,1
21003,21535,21535,54272.0,332.5373,16.8053,0.2720,19.0528,20.0008,0.774,0.059,...,-9999,J221009.3+165449,-31.180104,76.542726,0.327218,-0.000112,2.181500,-0.000118,0.000789,1
21004,21536,21536,54273.0,321.0614,-5.5885,0.4036,7.3319,20.0008,0.734,0.260,...,-9999,J212412.8-053647,-36.312474,46.861993,0.906488,0.000283,3.162386,0.000297,0.001527,1


---

In [48]:
pl = hp.read_map("HFI_SkyMap_353-psb-field-IQU_2048_R3.00_full.fits")
pl[pl<-1e23]=0

NSIDE = 2048
ORDERING = NESTED in fits file
INDXSCHM = IMPLICIT
Ordering converted to RING


In [49]:
shape,wcs = enmap.fullsky_geometry(res=5.0 * utils.arcmin,proj='car')

In [50]:
map_pix = reproject.enmap_from_healpix(pl,shape=shape,wcs=wcs)

Preparing SHT
T -> alm
float64 complex128
Computing pixel positions
Computing rotated positions
Projecting


---

In [51]:
signal = []
ras_temp = []
decs_temp = []

ras_fake = np.linspace(0,360,100)
decs_fake = np.linspace(-90,90,100)

N = len(ras_fake)
# N=10
stack_CMB_kSZ           = 0
stack_CMB_kSZ_deproject = 0
c = 0

In [52]:
def worker(i):
    for j in range(N):
        stamp = reproject.thumbnails(map_pix,coords = np.deg2rad([decs_fake[i], ras_fake[j]]),r=3*utils.arcmin)

        if stamp is None:
            continue
        elif stamp[0][0][0]==0.0:
            continue

        signal.append(sum(sum(stamp[0]))/9)
        ras_temp.append(ras_fake[j])
        decs_temp.append(decs_fake[i])
        
    return [signal, ras_temp, decs_temp]
        
# pool = mp.Pool(processes=8)
# for _ in tqdm(pool.imap_unordered(worker, range(N)), total=len(range(N))):
#     pass

with mp.Pool(processes = 1) as p:
    results = p.map(worker, [x for x in range(N)])

In [53]:
np.savetxt("results353_1.csv", 
           results[:10][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_2.csv", 
           results[10:20][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_3.csv", 
           results[20:30][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_4.csv", 
           results[30:40][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_5.csv", 
           results[40:50][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_6.csv", 
           results[50:60][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_7.csv", 
           results[60:70][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_8.csv", 
           results[70:80][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_9.csv", 
           results[80:90][0],
           delimiter =", ", 
           fmt ='% s')

np.savetxt("results353_10.csv", 
           results[90:100][0],
           delimiter =", ", 
           fmt ='% s')